In [1]:
import video_transect, misc
from pathlib import Path
import pandas as pd

In [2]:
# Paths
transect_dirs = [Path('/media/mha114/Massimal2/Massimal/Smola_Maholmen/DropCamTransects/20230621/T2'),
                 Path('/media/mha114/Massimal2/Massimal/Smola_Maholmen/DropCamTransects/20230621/T3'),
                 Path('/media/mha114/Massimal2/Massimal/Smola_Maholmen/DropCamTransects/20230621/T4')]
# transect_dirs = [Path('/media/mha114/Massimal2/Massimal/Smola_Maholmen/DropCamTransects/20230621/T2')] # Redo T2 after video rotation
csv_file  = Path('/media/mha114/Massimal2/Massimal/Smola_Maholmen/DropCamTransects/GPS_Tracks/skippo_tracks/20230621_skippo_tracks_maholmen.csv')

In [3]:
# Parameters
# sample_distance = 1.0 # Units defined by CRS, default is UTM with unit meters
# sample_distance = 10 # Units defined by CRS, default is UTM with unit meters
video_offset_sec = 210 #235 #223 #135 # Time offset between Skippo and GoPro

# Note
Skippo CSV file does not seem to include the first transect. Therefore, only transects 2-4 are included here.

In [4]:
# Read "Skippo" CSV file
skippo_gdf = video_transect.yx_csv_to_geodataframe(csv_file,time_column_name='DateTimeS')
skippo_gdf = skippo_gdf.dropna() # Drop points without timestamps (at end of file)
skippo_gdf = skippo_gdf.sort_values('Time')
#skippo_gdf = video_transect.filter_gdf_on_distance(skippo_gdf,sample_distance)
skippo_gdf

,Lat,Lng,Time,geometry
228,63.409889,7.73651,2023-06-21 11:12:25+00:00,POINT (7.73651 63.40989)
229,63.409882,7.73651,2023-06-21 11:12:27+00:00,POINT (7.73651 63.40988)
230,63.409870,7.73652,2023-06-21 11:12:29+00:00,POINT (7.73652 63.40987)
231,63.409859,7.73653,2023-06-21 11:12:31+00:00,POINT (7.73653 63.40986)
232,63.409851,7.73651,2023-06-21 11:12:33+00:00,POINT (7.73651 63.40985)
...,...,...,...,...
223,63.407139,7.74505,2023-06-21 13:34:09+00:00,POINT (7.74505 63.40714)
224,63.407150,7.74509,2023-06-21 13:34:11+00:00,POINT (7.74509 63.40715)
225,63.407181,7.74515,2023-06-21 13:34:13+00:00,POINT (7.74515 63.40718)
226,63.407242,7.74532,2023-06-21 13:34:15+00:00,POINT (7.74532 63.40724)


In [5]:
# NOTE: Do a "dry run" without extracting images and check that time offsets are correct

for transect_dir in transect_dirs:
    video_dir = transect_dir / 'Video'
    
    image_dir = transect_dir / 'ExtractedImages/Original'
    image_dir.mkdir(parents=True, exist_ok=True)
    print(f'{image_dir} created (if not created before)')

    gpkg_dir = transect_dir / 'ImagesGeopackage'
    gpkg_dir.mkdir(exist_ok=True)
    print(f'{gpkg_dir} created (if not created before)')

    gpkg_file = gpkg_dir / (transect_dir.stem + '_GeotaggedImages.gpkg')
    print(f'GPKG file name for saving results: {gpkg_file}')

    video_files = misc.file_pattern_search(str(video_dir), '*.[Mm][Pp]4')
    print(f'Found video files {[Path(vfile).name for vfile in video_files]}')
    video_data = video_transect.get_video_data(str(video_dir),tz='utc',video_time_offset=pd.Timedelta(hours=-2))
    #display(video_data)
    
    gdf = video_transect.prepare_gdf_with_video_data(skippo_gdf,video_data,video_offset_sec=video_offset_sec)
    # display(gdf)

    gdf = video_transect.extract_images_from_video(gdf,str(image_dir))
    gdf.to_file(gpkg_file, driver="GPKG")
    

/media/mha114/Massimal2/Massimal/Smola_Maholmen/DropCamTransects/20230621/T2/ExtractedImages/Original created (if not created before)
/media/mha114/Massimal2/Massimal/Smola_Maholmen/DropCamTransects/20230621/T2/ImagesGeopackage created (if not created before)
GPKG file name for saving results: /media/mha114/Massimal2/Massimal/Smola_Maholmen/DropCamTransects/20230621/T2/ImagesGeopackage/T2_GeotaggedImages.gpkg
Found video files ['GH010771.MP4', 'GH020771.MP4', 'GH030771.MP4', 'GH040771.MP4', 'GH050771.MP4']


100%|██████████| 1159/1159 [11:41<00:00,  1.65it/s]


/media/mha114/Massimal2/Massimal/Smola_Maholmen/DropCamTransects/20230621/T3/ExtractedImages/Original created (if not created before)
/media/mha114/Massimal2/Massimal/Smola_Maholmen/DropCamTransects/20230621/T3/ImagesGeopackage created (if not created before)
GPKG file name for saving results: /media/mha114/Massimal2/Massimal/Smola_Maholmen/DropCamTransects/20230621/T3/ImagesGeopackage/T3_GeotaggedImages.gpkg
Found video files ['GH010772.MP4', 'GH020772.MP4', 'GH030772.MP4']


100%|██████████| 553/553 [06:26<00:00,  1.43it/s]


/media/mha114/Massimal2/Massimal/Smola_Maholmen/DropCamTransects/20230621/T4/ExtractedImages/Original created (if not created before)
/media/mha114/Massimal2/Massimal/Smola_Maholmen/DropCamTransects/20230621/T4/ImagesGeopackage created (if not created before)
GPKG file name for saving results: /media/mha114/Massimal2/Massimal/Smola_Maholmen/DropCamTransects/20230621/T4/ImagesGeopackage/T4_GeotaggedImages.gpkg
Found video files ['GH010773.MP4', 'GH020773.MP4']


100%|██████████| 228/228 [02:15<00:00,  1.68it/s]
